In [132]:
!pip install quandl


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [133]:
!pip install numpy cython lunarcalendar convertdate holidays setuptools-git python-dateutil



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [134]:
!pip install prophet


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [135]:
!pip install ipywidgets plotly


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [136]:
from stocker import Stocker
import quandl
import yaml

import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta

# Using Stocker package to get data before 2018

In [137]:
with open("./api_key.yaml", 'r') as stream:
    try:
        data_loaded = yaml.load(stream, Loader=yaml.FullLoader)
    except yaml.YAMLError as exc:
        print(exc)

In [138]:
api_key = data_loaded['api_key']
quandl.ApiConfig.api_key = api_key

In [206]:
tickers = pd.read_csv('./tickers.csv')

In [207]:
tickers.columns

Index(['ticker'], dtype='object')

In [208]:
ticker_list = [i for i in tickers.ticker]

In [173]:
def agg_stock_data(ticker):
    try:
        stock_object = Stocker(ticker)
        stock_history_df = stock_object.stock
        stock_history_df.columns = [i.lower() for i in stock_history_df.columns]
        stock_history_df = (
            stock_history_df
            .pipe(lambda x: x.assign(ticker=ticker))
        )
        return stock_history_df
    except AttributeError as e:
        print(f"Error processing ticker {ticker}: {e}")
        return None

In [174]:
first = True
for tick in ticker_list:
    df = agg_stock_data(tick)
    if df is None:
        continue  # Skip this iteration if there was an error
    if first:
        if df.date.min().year <= 2006 and df.date.max().year == 2018:
            agg_df = df
            first = False
    else:
        if df.date.min().year <= 2006 and df.date.max().year == 2018:
            agg_df = pd.concat([agg_df, df], axis=0)

DDD Stocker Initialized. Data covers 1990-11-05 00:00:00 to 2018-03-27 00:00:00.
MMM Stocker Initialized. Data covers 1970-01-02 00:00:00 to 2018-03-07 00:00:00.
Error Retrieving Data.
(Status 404) (Quandl Error QECx02) You have submitted an incorrect Dataset code. Please check your Dataset codes and try again.
Error processing ticker WBAI: 'Stocker' object has no attribute 'stock'
Error Retrieving Data.
(Status 404) (Quandl Error QECx02) You have submitted an incorrect Dataset code. Please check your Dataset codes and try again.
Error processing ticker WUBA: 'Stocker' object has no attribute 'stock'
EGHT Stocker Initialized. Data covers 1997-07-02 00:00:00 to 2018-03-27 00:00:00.
AHC Stocker Initialized. Data covers 2008-02-11 00:00:00 to 2018-03-27 00:00:00.
AOS Stocker Initialized. Data covers 1984-09-07 00:00:00 to 2018-03-27 00:00:00.
ATEN Stocker Initialized. Data covers 2014-03-21 00:00:00 to 2018-03-27 00:00:00.
AIR Stocker Initialized. Data covers 1984-07-19 00:00:00 to 2018-0

In [175]:
agg_df.head()

,date,open,high,low,close,volume,ex-dividend,split ratio,adj. open,adj. high,adj. low,adj. close,adj. volume,ds,y,daily change,ticker
0,1990-11-05,3.88,4.25,3.25,4.25,34800.0,0.0,1.0,3.88,4.25,3.25,4.25,34800.0,1990-11-05,4.25,0.37,DDD
1,1990-11-06,3.50,4.25,3.50,3.62,21600.0,0.0,1.0,3.50,4.25,3.50,3.62,21600.0,1990-11-06,3.62,0.12,DDD
2,1990-11-07,3.50,4.00,3.50,4.00,20400.0,0.0,1.0,3.50,4.00,3.50,4.00,20400.0,1990-11-07,4.00,0.50,DDD
3,1990-11-08,4.00,4.00,3.75,4.00,16800.0,0.0,1.0,4.00,4.00,3.75,4.00,16800.0,1990-11-08,4.00,0.00,DDD
4,1990-11-09,4.00,4.00,3.50,3.50,300.0,0.0,1.0,4.00,4.00,3.50,3.50,300.0,1990-11-09,3.50,-0.50,DDD


In [176]:
agg_df.to_csv('./2012_2017_data.csv')

# Using Yahoo Finance Package

In [177]:
!pip install pandas_datareader fix_yahoo_finance


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [178]:
from pandas_datareader import data as pdr
import yfinance as yf

# yf.pdr_override()

In [209]:
import pandas as pd
import yfinance as yf

# Function to download data for the entire year
def download_data_for_year(ticker, year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    df = yf.download(ticker, start=start_date, end=end_date).reset_index()
    if not df.empty:
        df['ticker'] = ticker
    return df if not df.empty else None

# Download data for all tickers for the entire year 2018
df_2018 = pd.concat(
    [
        download_data_for_year(tick, 2018)
        for tick in ticker_list
        if download_data_for_year(tick, 2018) is not None
    ],
    axis=0
).reset_index(drop=True)

# Check if df_2018 was created and is not None
if not df_2018.empty:
    print("Final dataframe (df_2018) head:")
    print(df_2018.head(23))  # Print more rows to inspect
    # Save the dataframe to a CSV file
    # df_2018.to_csv('2018_stock_data.csv', index=False)
    # print("The data has been saved to '2018_stock_data.csv'.")
else:
    print("No valid data was collected for the given tickers.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WBAI']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WUBA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AHC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[****************

$EAI: possibly delisted; No price data found  (1d 2018-01-01 -> 2018-12-31)



[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ELC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1546232400")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ELJ']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['EMP']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2018-01-01 -> 2018-12-31)')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

$EMP: possibly delisted; No price data found  (1d 2018-01-01 -> 2018-12-31)



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NVST']: YFChartError("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1546232400")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%********************

$GRAF: possibly delisted; No price data found  (1d 2018-01-01 -> 2018-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['GRAM']: YFChartError("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1546232400")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*********************

$MDLX: possibly delisted; No price data found  (1d 2018-01-01 -> 2018-12-31)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MDLY']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MFAC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MDP']: YFTzMissingError

Final dataframe (df_2018) head:
         Date   Open   High    Low  Close  Adj Close   Volume ticker
0  2018-01-02   8.70   9.11   8.60   9.06       9.06  2499000    DDD
1  2018-01-03   9.12   9.37   9.10   9.31       9.31  1799100    DDD
2  2018-01-04   9.35   9.45   9.09   9.36       9.36  1982200    DDD
3  2018-01-05   9.42   9.53   9.22   9.39       9.39  1302700    DDD
4  2018-01-08   9.38   9.53   9.21   9.31       9.31  1898400    DDD
5  2018-01-09   9.31   9.57   9.23   9.45       9.45  1335200    DDD
6  2018-01-10   9.40   9.70   9.32   9.61       9.61  1302200    DDD
7  2018-01-11   9.65  10.30   9.63  10.30      10.30  2773900    DDD
8  2018-01-12  10.35  11.36  10.22  11.19      11.19  6275700    DDD
9  2018-01-16  11.21  11.80  10.47  10.71      10.71  6072200    DDD
10 2018-01-17  11.03  11.05   9.78   9.89       9.89  4445700    DDD
11 2018-01-18  10.03  10.68   9.82  10.55      10.55  4568900    DDD
12 2018-01-19  10.57  11.04  10.56  10.66      10.66  3208100    DDD
13

# Removing this part as this throwing errors

In [ ]:
# first = True
# for tick in ticker_list:
#     try:
#         start_2018_df = yf.download(tick, start="2018-01-03", end="2018-01-03").reset_index(drop=True)
#         start_2018_df = (
#             start_2018_df
#             .pipe(lambda x: x.assign(date=datetime.date(2018, 1, 3)))
#             .pipe(lambda x: x.assign(ticker='{}'.format(tick)))
#         )
#         end_2018_df = yf.download(tick, start="2018-12-29", end="2018-12-29").reset_index(drop=True)
#         end_2018_df = (
#             end_2018_df
#             .pipe(lambda x: x.assign(date=datetime.date(2018, 12, 29)))
#             .pipe(lambda x: x.assign(ticker='{}'.format(tick)))
#         )
#     except:
#         ValueError
#     if first is True:
#         df_2018 = pd.concat([start_2018_df, end_2018_df], join='outer', axis=0)
#         first = False
#     else:
#         intermediate_df = pd.concat([start_2018_df, end_2018_df], join='outer', axis=0)
#         df_2018 = pd.concat([df_2018, intermediate_df], join='outer', axis=0)

In [210]:
df_2018.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2018-01-02,8.70,9.11,8.60,9.06,9.06,2499000,DDD
1,2018-01-03,9.12,9.37,9.10,9.31,9.31,1799100,DDD
2,2018-01-04,9.35,9.45,9.09,9.36,9.36,1982200,DDD


In [211]:
df_2018.to_csv('./2018_data.csv')

In [212]:
data_2012_2017 = pd.read_csv('2012_2017_data.csv')
data_2018 = pd.read_csv('2018_data.csv')

# Combine both datasets into one DataFrame
agg_df = pd.concat([data_2012_2017, data_2018])

# Check the first few rows of the combined data to ensure everything is correct
agg_df.head()

# Save the aggregated data to a new CSV file
agg_df.to_csv('agg_data.csv', index=False)
